# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [3]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's. It is important to sort the unique values. Otherwise, the method is creating different results after each restart of the notebook. 

In [5]:
doi_counted = doi.value_counts().sort_index(ascending=True)
doi_counted

10.1001/jamainternmed.2020.1369       1
10.1001/jamanetworkopen.2020.16382    1
10.1001/jamanetworkopen.2020.17521    1
10.1001/jamanetworkopen.2020.20485    1
10.1001/jamanetworkopen.2020.24984    1
                                     ..
10.9745/ghsp-d-20-00115               1
10.9745/ghsp-d-20-00171               1
10.9745/ghsp-d-20-00218               1
10.9758/cpn.2020.18.4.607             1
10.9781/ijimai.2020.02.002            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API. (https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4)

In [6]:
#Adapted from https://github.com/ElsevierDev/elsapy/blob/master/exampleProg.py
def fetch_scopus_api(client, doi):
    """obtain additional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [7]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [8]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [9]:
return_example = fetch_scopus_api(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [10]:
df_current_extra_info = pd.DataFrame()
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    df_current_extra_info
except:
    print("The DataFrame is empty")
    #if the dataframe is not empty set the variable to show the dataframe

The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [11]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

41020

In [12]:
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': 'Palo Alto', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083266658', ..."
1,"[{'affiliation-city': 'Seattle', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
2,"[{'affiliation-city': 'Cambridge', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
3,"[{'affiliation-city': 'Madison', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '9',..."
4,"[{'affiliation-city': 'Los Angeles', 'affilnam...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
...,...,...
41015,None,None
41016,None,None
41017,None,None
41018,None,None


In [13]:
def contains_only_None(dic):
    """
    This functions inspects an dictionary and returns True if it solely contains None values
    """
    return len(dic) == sum(value == None for value in dict_new_extra_info.values())

In [14]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    if contains_only_None(dic):
        placeholder_entries = pd.DataFrame(np.empty((len(dict_new_extra_info),2),dtype=object),columns=['affiliation','coredata'], index=dict_new_extra_info.keys())
        df_newly_fetched_transposed = placeholder_entries
        print(placeholder_entries)
    else:
        #Prior appending, the dictionary is transformed to a DataFrame
        df_newly_fetched = pd.DataFrame(dic)
        #For readability, the DataFrame is transposed
        df_newly_fetched_transposed = df_newly_fetched.T
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row.affiliation is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [15]:
def store_df_columns(df):
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    ser_affiliation.to_pickle('extra_info_affiliation_CS.pkl')
    ser_coredata.to_pickle('extra_info_coredata_CS.pkl')
    return ser_affiliation, ser_coredata

In [16]:
# placeholder_entries = pd.DataFrame(np.empty((4,2),dtype=object),columns=['affiliation','coredata'])

In [17]:
# placeholder_entries

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [ ]:
%%time
dict_new_extra_info = dict()
len_dois = len(doi_counted)
def trigger_fetching():
    threshold = 0 
    i = len_df_current_extra_info
    while i < len_dois: #-> upto modified, normally len_dois
        dict_new_extra_info[i] = fetch_scopus_api(client, doi_counted.index[i])
        print("Position fetched: " + str(i) + " -> " +  doi_counted.index[i])
        i = i + 1 
        threshold = threshold + 1
        if threshold > 99:
            df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
            stored_series = store_df_columns(df_combined_extra_info)
            threshold = 0
            print("batch saved")
trigger_fetching()

Position fetched: 41020 -> 10.1101/2020.03.18.20037101
Position fetched: 41021 -> 10.1101/2020.03.18.20037473
Position fetched: 41022 -> 10.1101/2020.03.18.20037994
Position fetched: 41023 -> 10.1101/2020.03.18.20038018
Position fetched: 41024 -> 10.1101/2020.03.18.20038026
Position fetched: 41025 -> 10.1101/2020.03.18.20038059
Position fetched: 41026 -> 10.1101/2020.03.18.20038117
Position fetched: 41027 -> 10.1101/2020.03.18.20038224
Position fetched: 41028 -> 10.1101/2020.03.18.20038455
Position fetched: 41029 -> 10.1101/2020.03.18.20038570
Position fetched: 41030 -> 10.1101/2020.03.18.20038612
Position fetched: 41031 -> 10.1101/2020.03.18.20038638
Position fetched: 41032 -> 10.1101/2020.03.18.996454
Position fetched: 41033 -> 10.1101/2020.03.19.20027078
Position fetched: 41034 -> 10.1101/2020.03.19.20034124
Position fetched: 41035 -> 10.1101/2020.03.19.20038315
Position fetched: 41036 -> 10.1101/2020.03.19.20038539
Position fetched: 41037 -> 10.1101/2020.03.19.20038752
Position fet

Position fetched: 41164 -> 10.1101/2020.03.25.20043711
Position fetched: 41165 -> 10.1101/2020.03.25.20043745
Position fetched: 41166 -> 10.1101/2020.03.25.20043828
Position fetched: 41167 -> 10.1101/2020.03.25.20043877
Position fetched: 41168 -> 10.1101/2020.03.25.20043893
Position fetched: 41169 -> 10.1101/2020.03.25.998963
Position fetched: 41170 -> 10.1101/2020.03.26.009001
Position fetched: 41171 -> 10.1101/2020.03.26.009209
Position fetched: 41172 -> 10.1101/2020.03.26.009605
Position fetched: 41173 -> 10.1101/2020.03.26.009662
Position fetched: 41174 -> 10.1101/2020.03.26.009803
Position fetched: 41175 -> 10.1101/2020.03.26.009993
Position fetched: 41176 -> 10.1101/2020.03.26.010165
Position fetched: 41177 -> 10.1101/2020.03.26.010322
Position fetched: 41178 -> 10.1101/2020.03.26.20040709
Position fetched: 41179 -> 10.1101/2020.03.26.20041426
Position fetched: 41180 -> 10.1101/2020.03.26.20043042
Position fetched: 41181 -> 10.1101/2020.03.26.20044057
Position fetched: 41182 -> 1

Position fetched: 41309 -> 10.1101/2020.04.01.20049668
Position fetched: 41310 -> 10.1101/2020.04.01.20049684
Position fetched: 41311 -> 10.1101/2020.04.01.20049692
Position fetched: 41312 -> 10.1101/2020.04.01.20049759
Position fetched: 41313 -> 10.1101/2020.04.01.20049825
Position fetched: 41314 -> 10.1101/2020.04.01.20049908
Position fetched: 41315 -> 10.1101/2020.04.01.20049973
Position fetched: 41316 -> 10.1101/2020.04.01.20050039
Position fetched: 41317 -> 10.1101/2020.04.01.20050054
Position fetched: 41318 -> 10.1101/2020.04.01.20050138
Position fetched: 41319 -> 10.1101/2020.04.01.20050203
Position fetched: 41320 -> 10.1101/2020.04.01.20050237
Position fetched: 41321 -> 10.1101/2020.04.01.20050393
Position fetched: 41322 -> 10.1101/2020.04.01.20050443
      affiliation coredata
41020        None     None
41021        None     None
41022        None     None
41023        None     None
41024        None     None
...           ...      ...
41318        None     None
41319        N

Position fetched: 41447 -> 10.1101/2020.04.06.20054841
Position fetched: 41448 -> 10.1101/2020.04.06.20054882
Position fetched: 41449 -> 10.1101/2020.04.06.20054890
Position fetched: 41450 -> 10.1101/2020.04.06.20055103
Position fetched: 41451 -> 10.1101/2020.04.06.20055194
Position fetched: 41452 -> 10.1101/2020.04.06.20055228
Position fetched: 41453 -> 10.1101/2020.04.06.20055244
Position fetched: 41454 -> 10.1101/2020.04.06.20055285
Position fetched: 41455 -> 10.1101/2020.04.06.20055327
Position fetched: 41456 -> 10.1101/2020.04.06.20055368
Position fetched: 41457 -> 10.1101/2020.04.06.20055384
Position fetched: 41458 -> 10.1101/2020.04.06.20055426
Position fetched: 41459 -> 10.1101/2020.04.06.20055467
Position fetched: 41460 -> 10.1101/2020.04.06.20055582
Position fetched: 41461 -> 10.1101/2020.04.06.20055616
Position fetched: 41462 -> 10.1101/2020.04.06.20055624
Position fetched: 41463 -> 10.1101/2020.04.06.20055657
Position fetched: 41464 -> 10.1101/2020.04.06.20055699
Position f

Position fetched: 41591 -> 10.1101/2020.04.10.20060673
Position fetched: 41592 -> 10.1101/2020.04.10.20060681
Position fetched: 41593 -> 10.1101/2020.04.10.20060699
Position fetched: 41594 -> 10.1101/2020.04.10.20060707
Position fetched: 41595 -> 10.1101/2020.04.10.20060830
Position fetched: 41596 -> 10.1101/2020.04.10.20060962
Position fetched: 41597 -> 10.1101/2020.04.10.20061036
Position fetched: 41598 -> 10.1101/2020.04.10.20061069
Position fetched: 41599 -> 10.1101/2020.04.10.20061176
Position fetched: 41600 -> 10.1101/2020.04.10.20061192
Position fetched: 41601 -> 10.1101/2020.04.10.20061226
Position fetched: 41602 -> 10.1101/2020.04.10.20061325
Position fetched: 41603 -> 10.1101/2020.04.10.20061358
Position fetched: 41604 -> 10.1101/2020.04.10.20061374
Position fetched: 41605 -> 10.1101/2020.04.11.037093
Position fetched: 41606 -> 10.1101/2020.04.11.037358
Position fetched: 41607 -> 10.1101/2020.04.11.037473
Position fetched: 41608 -> 10.1101/2020.04.11.20054643
Position fetched

Position fetched: 41729 -> 10.1101/2020.04.15.20057786
Position fetched: 41730 -> 10.1101/2020.04.15.20062752
Position fetched: 41731 -> 10.1101/2020.04.15.20063107
Position fetched: 41732 -> 10.1101/2020.04.15.20064485
Position fetched: 41733 -> 10.1101/2020.04.15.20064931
Position fetched: 41734 -> 10.1101/2020.04.15.20065946
Position fetched: 41735 -> 10.1101/2020.04.15.20066100
Position fetched: 41736 -> 10.1101/2020.04.15.20066183
Position fetched: 41737 -> 10.1101/2020.04.15.20066407
Position fetched: 41738 -> 10.1101/2020.04.15.20066431
Position fetched: 41739 -> 10.1101/2020.04.15.20066472
Position fetched: 41740 -> 10.1101/2020.04.15.20066548
Position fetched: 41741 -> 10.1101/2020.04.15.20066613
Position fetched: 41742 -> 10.1101/2020.04.15.20066712
Position fetched: 41743 -> 10.1101/2020.04.15.20066746
Position fetched: 41744 -> 10.1101/2020.04.15.20066845
Position fetched: 41745 -> 10.1101/2020.04.15.20066860
Position fetched: 41746 -> 10.1101/2020.04.15.20066894
Position f

Position fetched: 41873 -> 10.1101/2020.04.20.048066
Position fetched: 41874 -> 10.1101/2020.04.20.049866
Position fetched: 41875 -> 10.1101/2020.04.20.049924
Position fetched: 41876 -> 10.1101/2020.04.20.050039
Position fetched: 41877 -> 10.1101/2020.04.20.050088
Position fetched: 41878 -> 10.1101/2020.04.20.050138
Position fetched: 41879 -> 10.1101/2020.04.20.050393
Position fetched: 41880 -> 10.1101/2020.04.20.051011
Position fetched: 41881 -> 10.1101/2020.04.20.051219
Position fetched: 41882 -> 10.1101/2020.04.20.051581
Position fetched: 41883 -> 10.1101/2020.04.20.051656
Position fetched: 41884 -> 10.1101/2020.04.20.052019
Position fetched: 41885 -> 10.1101/2020.04.20.052068
Position fetched: 41886 -> 10.1101/2020.04.20.052159
Position fetched: 41887 -> 10.1101/2020.04.20.20064105
Position fetched: 41888 -> 10.1101/2020.04.20.20065953
Position fetched: 41889 -> 10.1101/2020.04.20.20067512
Position fetched: 41890 -> 10.1101/2020.04.20.20068676
Position fetched: 41891 -> 10.1101/202

The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [ ]:
def save_new_extra_info(len_df_current_extra_info, upto):
    """
    This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
    As a result, this function returns a range of valid entries up to the given parameter. 
    """
    dict_new_extra_info_saver = dict()
    i = len_df_current_extra_info
    while i < upto:
        #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
        dict_new_extra_info_saver[i] = dict_new_extra_info[i]
        i = i + 1 
    return dict_new_extra_info_saver

The existing and newly fetched information are combined into one DataFrame. 

In [ ]:
# df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
df_combined_extra_info

In [ ]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [ ]:
# stored_series = store_df_columns(df_combined_extra_info)
# stored_series[0]

In [ ]:
# stored_series[1]

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [ ]:
# len_data = len(stored_series[0])
# len_data 

In [ ]:
# ser_doi = pd.Series(doi_counted.index[:len_data])
# ser_doi

In [ ]:
# df_current_extra_info_checker = df_combined_extra_info
# df_current_extra_info_checker['doi'] = ser_doi

In [ ]:
# %%time
# len_df_current_extra_info_checker = len(df_current_extra_info_checker)
# dict_new_extra_info_checker = dict()
# i = 0 
# while i < len_df_current_extra_info_checker: ###################################################### 
#     if df_current_extra_info_checker['affiliation'][i] == None:
#         dict_new_extra_info_checker[i] = fetch_scopus_api(client, ser_doi[i])
#         print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
#     i = i + 1    

In [ ]:
# dict_new_extra_info_checker
# -> check if at least one value is not None -> otherwise the process is finished here

In [ ]:
# len(dict_new_extra_info_checker)

In [ ]:
# df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
# df_combined_extra_info_fetched_again

In [ ]:
# store_df_columns(df_combined_extra_info_fetched_again)

In [ ]:
# df_combined_extra_info_fetched_again['check_doi'] = ser_doi
# df_combined_extra_info_fetched_again.head(30)